In [1]:
import os
import json
import pprint
import re
from PIL import Image as PILImage
from Prompts import *
import pickle as pkl
import yaml
from utils.config import *
from utils.models import *
import copy
import subprocess

In [2]:
model = GPTModel(config = dict(
    MODEL_NAME = MODEL
))

In [3]:
model.clear_assistant_files()

In [22]:
scenario_desc = "The delivery bot needs to transport a box through a narrow passageway in the warehouse. As it navigates the passageway, it encounters a warehouse employee who signals the robot to 'WAIT'."
num_humans = 1

In [23]:
with open(os.path.join('locations',LOCATION,'scene_graph.json'),'r') as f:
    scene_graph = json.load(f)
scgraph = utils.SceneGraph(scene_graph)
node_types = []
for node in scgraph.get_parent_nodes():
    if scgraph.graph.nodes[node]['type'] not in node_types:
        node_types.append(scgraph.graph.nodes[node]['type'])
print(node_types)

['PASSAGEWAY', 'CORNER', 'INTERSECTION', 'OPEN AREA']


In [24]:
flocationQ= FLocationQuery(
    scene_graph_img = None
)
flq_full_text_prompt = flocationQ.get_full_prompt(
    sc_desc = scenario_desc,
    scene_graph_nodes = ','.join(node_types)+',child',
    num_humans = num_humans,
)

In [25]:
### Use get files
sg_json_file = model.get_files_by_name('scene_graph.json')
sg_img_file = model.get_files_by_name('scene_graph.png')
if sg_json_file!=None:
    print("Loaded Json SG file")
else:
    print("Creating JSON SG file")
    sg_json_file = model.create_file(os.path.join('locations',LOCATION,'scene_graph.json'), purpose = 'assistants')

if sg_img_file!=None:
    print("Loaded Image SG file")
else:
    print("Creating Image SG file")
    sg_img_file = model.create_file(os.path.join('locations',LOCATION,'scene_graph.png'), purpose = 'vision')


Loaded Json SG file
Loaded Image SG file


In [26]:
location_content = [
            {
             "type":"text",
             "text":flq_full_text_prompt
            },
            {
                "type":"image_file",
                "image_file":{
                    "file_id":sg_img_file.id,
                    "detail":"high"
                }
            },
        ]
location_attachments = [
             {"file_id":sg_json_file.id,"tools":[{"type":"file_search"}]}
        ]

In [27]:
# get assistant
location_assistant = model.get_assistant_by_name('location_scenario_assistant')
if location_assistant == None:
    print("Creating New Assistant")
    #create assistant
    location_assistant = model.client.beta.assistants.create(
        instructions="You are an expert floor planner and a software engineer. When asked a question, always return an answer that is fully parseable with python json.loads",
        name="location_scenario_assistant",
        tools=[{"type": "file_search"}],
        model=model.model_name,
    )
else:
    print("Loaded Assistant")

Loaded Assistant


In [28]:
thread_id, run_id = model.create_and_run_thread(
    content = location_content,
    attachments = location_attachments,
    assistant_id = location_assistant.id
)

In [29]:
response = model.poll_run_result(thread_id,run_id)

Run in_progress
Run in_progress
Run in_progress
Run in_progress
Run in_progress
Run in_progress
Run in_progress
Run in_progress
Run in_progress
Run in_progress
Run in_progress
Run in_progress
Run in_progress
Run in_progress
Run in_progress
Run in_progress
Run in_progress
Run completed!
```json
{
    "TRAJECTORIES": {   
        "Robot": [22, 21, 23, 24],
        "Human 1": [21, 22]
    },
    "Group IDs": {
        "Human 1": -1
    },
    "REASONING": "The robot starts at node 22, which is at the entrance of the passageway. It moves through the passageway nodes 21 and 23, experiencing a delay at node 23 due to crossing paths with Human 1. Human 1 starts at node 21, signaling 'WAIT' to the robot at node 22 before proceeding to node 22. This setup creates the required scenario."
}
```
completed
